# Function Call（工具调用）测试

本 Notebook 演示智谱 AI 的 Function Call 功能，包括：
- 简单函数调用
- 数学计算工具
- 多工具组合
- 多轮对话
- 网络搜索工具
- 错误处理

## 1. 环境配置

In [ ]:
# 安装依赖（如未安装）
# !pip install openai

In [ ]:
import os
import json
from openai import OpenAI

# API 配置
API_KEY = "f7ec61a7856a4f4389e037eb588c69df.0pZgcFlTCk9GHCXM"
BASE_URL = "https://open.bigmodel.cn/api/paas/v4"
MODEL = "glm-4-flash-250414"  # 支持工具调用的模型

# 初始化客户端
client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

print(f"✅ 客户端初始化成功")
print(f"🤖 使用模型: {MODEL}")

## 2. 简单函数调用测试 - 天气查询

In [ ]:
# 定义天气查询函数
def get_weather(city: str) -> str:
    """模拟获取指定城市的天气信息"""
    weather_db = {
        "北京": {"temperature": 25, "condition": "晴天", "humidity": "45%"},
        "上海": {"temperature": 28, "condition": "多云", "humidity": "65%"},
        "广州": {"temperature": 30, "condition": "小雨", "humidity": "80%"},
        "深圳": {"temperature": 31, "condition": "阴天", "humidity": "75%"},
        "杭州": {"temperature": 26, "condition": "晴转多云", "humidity": "55%"},
    }
    
    if city in weather_db:
        data = weather_db[city]
        return f"{city}天气：{data['condition']}，温度{data['temperature']}°C，湿度{data['humidity']}"
    else:
        return f"抱歉，暂无{city}的天气数据"

# 测试函数
print(get_weather("北京"))
print(get_weather("上海"))

In [ ]:
# 工具定义
weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "获取指定城市的当前天气信息，包括温度、天气状况和湿度",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "城市名称，如北京、上海、广州等"
                }
            },
            "required": ["city"]
        }
    }
}

# 第一次调用：让模型判断是否需要工具
messages = [{"role": "user", "content": "北京今天天气怎么样？"}]

response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=[weather_tool],
    tool_choice="auto"
)

message = response.choices[0].message
print("模型响应:")
print(f"  内容: {message.content}")
print(f"  工具调用: {message.tool_calls}")

In [ ]:
# 如果模型请求调用工具，执行并回传结果
if message.tool_calls:
    # 添加模型的工具调用请求
    messages.append(message)
    
    # 执行工具调用
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        print(f"🔧 执行函数: {function_name}")
        print(f"📋 参数: {arguments}")
        
        # 调用实际函数
        result = get_weather(**arguments)
        print(f"✅ 结果: {result}")
        
        # 回传结果
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result
        })
    
    # 第二次调用：获取最终回复
    final_response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=[weather_tool]
    )
    
    print("\n📝 最终回复:")
    print(final_response.choices[0].message.content)
else:
    print("模型直接回复:")
    print(message.content)

## 3. 数学计算工具测试

In [ ]:
# 定义安全计算器函数
def calculate(expression: str) -> str:
    """
    安全执行数学计算
    只允许基本数学运算符：+ - * / ** % ( )
    """
    try:
        # 白名单字符检查
        allowed_chars = set('0123456789+-*/.() %**')
        cleaned = expression.replace('**', '').replace(' ', '')
        
        if not all(c in allowed_chars for c in cleaned):
            return "错误：表达式包含非法字符"
        
        # 安全计算
        result = eval(expression, {"__builtins__": {}}, {})
        return f"{expression} = {result}"
    except ZeroDivisionError:
        return "错误：除数不能为零"
    except Exception as e:
        return f"计算错误: {str(e)}"

# 测试计算器
print(calculate("123 + 456"))
print(calculate("100 / 4"))
print(calculate("2 ** 10"))

In [ ]:
# 计算器工具定义
calculator_tool = {
    "type": "function",
    "function": {
        "name": "calculate",
        "description": "执行数学计算，支持加减乘除、幂运算和括号，如 1+2*3, (100-50)/2, 2**10",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "数学表达式，例如：123*456, (100+200)/3"
                }
            },
            "required": ["expression"]
        }
    }
}

# 测试计算请求
def run_tool_conversation(user_input: str, tools: list) -> str:
    """运行带工具调用的对话"""
    messages = [{"role": "user", "content": user_input}]
    
    # 第一次调用
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    message = response.choices[0].message
    
    # 处理工具调用
    if message.tool_calls:
        messages.append(message)
        
        for tool_call in message.tool_calls:
            func_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)
            
            if func_name == "calculate":
                result = calculate(**args)
            elif func_name == "get_weather":
                result = get_weather(**args)
            else:
                result = f"未知函数: {func_name}"
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result)
            })
        
        # 获取最终回复
        final = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )
        return final.choices[0].message.content
    
    return message.content

# 测试计算
result = run_tool_conversation(
    "计算 1234 * 5678 等于多少？", 
    [calculator_tool]
)
print(f"📝 结果: {result}")

## 4. 多个工具组合测试

In [ ]:
# 组合工具列表
combined_tools = [weather_tool, calculator_tool]

# 测试需要同时使用多个工具的场景
test_queries = [
    "北京天气怎么样？顺便计算一下25乘以40等于多少？",
    "如果我在上海，气温是28度，那华氏度是多少？（计算 28 * 9/5 + 32）",
    "比较一下北京和广州的天气温度差",
]

for query in test_queries:
    print(f"\n{'='*50}")
    print(f"🙋 用户: {query}")
    result = run_tool_conversation(query, combined_tools)
    print(f"🤖 助手: {result}")

## 5. 工具调用 + 多轮对话测试

In [ ]:
# 带历史的工具调用对话
class ToolChat:
    def __init__(self, client, model, tools):
        self.client = client
        self.model = model
        self.tools = tools
        self.messages = []
        self.available_functions = {
            "get_weather": get_weather,
            "calculate": calculate
        }
    
    def send_message(self, user_input: str) -> str:
        """发送消息并获取回复"""
        self.messages.append({"role": "user", "content": user_input})
        
        # 第一次调用
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            tools=self.tools,
            tool_choice="auto"
        )
        
        message = response.choices[0].message
        
        # 处理工具调用
        if message.tool_calls:
            self.messages.append(message)
            
            for tool_call in message.tool_calls:
                func_name = tool_call.function.name
                args = json.loads(tool_call.function.arguments)
                
                if func_name in self.available_functions:
                    result = self.available_functions[func_name](**args)
                else:
                    result = f"未知函数: {func_name}"
                
                self.messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(result)
                })
            
            # 获取最终回复
            final = self.client.chat.completions.create(
                model=self.model,
                messages=self.messages,
                tools=self.tools
            )
            reply = final.choices[0].message.content
            self.messages.append(final.choices[0].message)
        else:
            reply = message.content
            self.messages.append(message)
        
        return reply

# 创建聊天实例
chat = ToolChat(client, MODEL, combined_tools)

# 多轮对话测试
conversations = [
    "你好，能帮我查一下北京的天气吗？",
    "那上海呢？",  # 上下文：用户想查天气
    "这两个城市温度差多少？",  # 需要计算温差
    "谢谢！"
]

for msg in conversations:
    print(f"\n🙋 用户: {msg}")
    reply = chat.send_message(msg)
    print(f"🤖 助手: {reply}")

## 6. 网络搜索工具测试

In [ ]:
# 网络搜索工具
web_search_tool = {
    "type": "web_search",
    "web_search": {
        "enable": True
    }
}

# 测试网络搜索
search_queries = [
    "今天有什么科技新闻？",
    "最新的人工智能发展趋势是什么？",
]

for query in search_queries[:1]:  # 只测试第一个，避免过多请求
    print(f"\n🙋 用户: {query}")
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": query}],
        tools=[web_search_tool]
    )
    
    message = response.choices[0].message
    
    # 检查搜索结果
    print(f"🤖 助手: {message.content[:500]}..." if len(message.content) > 500 else f"🤖 助手: {message.content}")
    
    # 如果有搜索结果引用
    if hasattr(message, 'tool_calls') and message.tool_calls:
        print(f"\n📚 搜索工具调用: {message.tool_calls}")

## 7. 工具调用失败处理测试

In [ ]:
# 模拟会失败的函数
def unstable_function(city: str) -> str:
    """不稳定的天气查询（模拟失败）"""
    import random
    if random.random() < 0.5:
        return f"{city}天气查询失败：服务暂时不可用"
    return f"{city}天气：晴天"

# 添加不稳定函数到工具
unstable_tool = {
    "type": "function",
    "function": {
        "name": "unstable_function",
        "description": "获取天气（可能失败）",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "城市名"}
            },
            "required": ["city"]
        }
    }
}

def run_with_error_handling(user_input: str) -> str:
    """带错误处理的工具调用"""
    messages = [{"role": "user", "content": user_input}]
    available_funcs = {
        "get_weather": get_weather,
        "calculate": calculate,
        "unstable_function": unstable_function
    }
    tools = [weather_tool, calculator_tool, unstable_tool]
    
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        message = response.choices[0].message
        
        if message.tool_calls:
            messages.append(message)
            
            for tool_call in message.tool_calls:
                func_name = tool_call.function.name
                args = json.loads(tool_call.function.arguments)
                
                try:
                    if func_name in available_funcs:
                        result = available_funcs[func_name](**args)
                        # 检查结果是否包含错误
                        if "错误" in str(result) or "失败" in str(result):
                            print(f"⚠️ 工具返回错误: {result}")
                    else:
                        result = f"错误：未知函数 {func_name}"
                except Exception as e:
                    result = f"工具执行错误: {str(e)}"
                    print(f"❌ 异常: {e}")
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(result)
                })
            
            final = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                tools=tools
            )
            return final.choices[0].message.content
        
        return message.content
        
    except Exception as e:
        return f"系统错误: {str(e)}"

# 测试错误处理
print("测试错误处理机制:")
for i in range(3):
    print(f"\n--- 第 {i+1} 次尝试 ---")
    result = run_with_error_handling("查询北京的天气")
    print(f"结果: {result}")

## 8. 完整的 Agent 流程测试

In [ ]:
# 定义更多工具函数
def get_current_time() -> str:
    """获取当前时间"""
    from datetime import datetime
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    """货币转换（模拟汇率）"""
    rates = {
        ("USD", "CNY"): 7.2,
        ("CNY", "USD"): 0.139,
        ("EUR", "CNY"): 7.8,
        ("CNY", "EUR"): 0.128,
    }
    key = (from_currency.upper(), to_currency.upper())
    if key in rates:
        result = amount * rates[key]
        return f"{amount} {from_currency.upper()} = {result:.2f} {to_currency.upper()}"
    return f"抱歉，暂不支持 {from_currency} 到 {to_currency} 的转换"

# 完整的工具集
agent_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取指定城市的天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "城市名称"}
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "执行数学计算",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string", "description": "数学表达式"}
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "获取当前的日期和时间",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "convert_currency",
            "description": "货币转换，支持 USD/CNY/EUR",
            "parameters": {
                "type": "object",
                "properties": {
                    "amount": {"type": "number", "description": "金额"},
                    "from_currency": {"type": "string", "description": "源货币，如USD"},
                    "to_currency": {"type": "string", "description": "目标货币，如CNY"}
                },
                "required": ["amount", "from_currency", "to_currency"]
            }
        }
    }
]

agent_functions = {
    "get_weather": get_weather,
    "calculate": calculate,
    "get_current_time": get_current_time,
    "convert_currency": convert_currency
}

print("✅ Agent 工具集准备完成")
print(f"🛠️ 可用工具: {list(agent_functions.keys())}")

In [ ]:
class Agent:
    """
    完整 Agent 实现
    支持多工具调用、多轮对话、错误处理
    """
    def __init__(self, client, model, tools, functions):
        self.client = client
        self.model = model
        self.tools = tools
        self.functions = functions
        self.messages = []
        self.max_iterations = 5  # 防止无限循环
    
    def reset(self):
        """重置对话历史"""
        self.messages = []
    
    def run(self, user_input: str, verbose: bool = False) -> str:
        """
        执行 Agent 流程
        
        Args:
            user_input: 用户输入
            verbose: 是否打印详细日志
        """
        self.messages.append({"role": "user", "content": user_input})
        
        if verbose:
            print(f"\n🙋 用户: {user_input}")
        
        iteration = 0
        while iteration < self.max_iterations:
            iteration += 1
            
            # 调用模型
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.messages,
                tools=self.tools,
                tool_choice="auto"
            )
            
            message = response.choices[0].message
            
            # 检查是否需要工具调用
            if not message.tool_calls:
                # 不需要工具，直接返回
                self.messages.append(message)
                if verbose:
                    print(f"🤖 助手: {message.content}")
                return message.content
            
            # 需要工具调用
            self.messages.append(message)
            
            if verbose:
                print(f"🔧 迭代 {iteration}: 检测到 {len(message.tool_calls)} 个工具调用")
            
            # 执行所有工具调用
            for tool_call in message.tool_calls:
                func_name = tool_call.function.name
                
                try:
                    args = json.loads(tool_call.function.arguments)
                except json.JSONDecodeError as e:
                    result = f"参数解析错误: {str(e)}"
                    if verbose:
                        print(f"  ❌ {func_name}: 参数解析失败")
                else:
                    if func_name in self.functions:
                        try:
                            result = self.functions[func_name](**args)
                            if verbose:
                                print(f"  ✅ {func_name}({args}) -> {result}")
                        except Exception as e:
                            result = f"执行错误: {str(e)}"
                            if verbose:
                                print(f"  ❌ {func_name}: {e}")
                    else:
                        result = f"未知函数: {func_name}"
                        if verbose:
                            print(f"  ⚠️ 未知函数: {func_name}")
                
                # 添加工具结果
                self.messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(result)
                })
        
        # 达到最大迭代次数
        return "错误：达到最大迭代次数"

# 创建 Agent
agent = Agent(client, MODEL, agent_tools, agent_functions)

# 测试复杂场景
test_cases = [
    "现在几点了？北京天气怎么样？",
    "100美元能换多少人民币？",
    "如果我在上海，气温28度，100美元能买多少杯5元一杯的奶茶？",
]

for test in test_cases:
    print(f"\n{'='*60}")
    result = agent.run(test, verbose=True)
    agent.reset()  # 重置历史，每个测试独立

## 9. 错误处理测试

In [ ]:
# 测试各种错误场景
error_tests = [
    {
        "name": "无效参数",
        "setup": lambda: None,
        "test": lambda: calculate("invalid_expression!!!")
    },
    {
        "name": "除零错误",
        "setup": lambda: None,
        "test": lambda: calculate("100 / 0")
    },
    {
        "name": "不存在的城市",
        "setup": lambda: None,
        "test": lambda: get_weather("不存在的城市XYZ")
    },
    {
        "name": "不支持的货币",
        "setup": lambda: None,
        "test": lambda: convert_currency(100, "XYZ", "ABC")
    },
]

print("错误处理测试结果:")
print("=" * 50)

for test in error_tests:
    test["setup"]()
    try:
        result = test["test"]()
        print(f"\n📋 {test['name']}:")
        print(f"   结果: {result}")
    except Exception as e:
        print(f"\n📋 {test['name']}:")
        print(f"   异常: {type(e).__name__}: {e}")

In [ ]:
# API 错误处理测试
def test_api_errors():
    """测试 API 相关错误"""
    
    # 1. 测试无效 API Key
    print("\n1. 测试无效 API Key:")
    try:
        bad_client = OpenAI(api_key="invalid_key", base_url=BASE_URL)
        response = bad_client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": "你好"}]
        )
        print("   意外成功")
    except Exception as e:
        print(f"   ✅ 正确捕获错误: {type(e).__name__}")
    
    # 2. 测试无效模型
    print("\n2. 测试无效模型:")
    try:
        response = client.chat.completions.create(
            model="invalid-model-name",
            messages=[{"role": "user", "content": "你好"}]
        )
        print("   意外成功")
    except Exception as e:
        print(f"   ✅ 正确捕获错误: {type(e).__name__}")
    
    # 3. 测试无效工具定义
    print("\n3. 测试无效工具定义:")
    try:
        invalid_tool = {
            "type": "function",
            "function": {
                "name": "test",
                # 缺少 description 和 parameters
            }
        }
        response = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": "测试"}],
            tools=[invalid_tool]
        )
        print("   请求已发送（可能使用默认行为）")
    except Exception as e:
        print(f"   ✅ 正确捕获错误: {type(e).__name__}")

test_api_errors()

## 10. 总结

In [ ]:
print("""
╔═══════════════════════════════════════════════════════════════╗
║                  Function Call 测试总结                      ║
╠═══════════════════════════════════════════════════════════════╣
║                                                               ║
║  ✅ 已测试功能：                                              ║
║     1. 简单函数调用（天气查询）                               ║
║     2. 数学计算工具                                           ║
║     3. 多工具组合使用                                         ║
║     4. 多轮对话支持                                           ║
║     5. 网络搜索工具                                           ║
║     6. 工具调用失败处理                                       ║
║     7. 完整 Agent 流程                                        ║
║     8. 错误处理机制                                           ║
║                                                               ║
║  📋 关键要点：                                                ║
║     • 工具定义需要清晰的 description                          ║
║     • 完整调用流程：定义→判断→执行→回传→回复                 ║
║     • 支持 function/web_search/retrieval 三种类型             ║
║     • 多轮对话需要保留完整消息历史                            ║
║     • 错误处理是生产环境必需的                                ║
║                                                               ║
║  🔧 支持的模型：                                              ║
║     • glm-4.7-flash                                           ║
║     • glm-4-flash-250414                                      ║
║     • glm-4.6v-flash (思考模型)                               ║
║                                                               ║
╚═══════════════════════════════════════════════════════════════╝
""")